Очистка данных

In [12]:
import json
import re
import unicodedata
import pandas as pd
import requests

Сначала пишем функции для очистки текста и удаления невидимых символов

In [13]:
def remove_control_symbols(text):
    # Убираем невидимые символы категории Cf (мягкий перенос U+00AD, zero-width space и т.п.).
    if not isinstance(text, str):
        return text
    return ''.join(ch for ch in text if unicodedata.category(ch) != 'Cf')

def split_on_case(text):
    # Вставляем пробел перед заглавной буквой после строчной (для кириллицы и латиницы), чтобы разорвать «склеившиеся» слова
    if not isinstance(text, str):
        return text
    text = re.sub(r'(?<=[а-яё])(?=[А-ЯЁ])', ' ', text)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    return text

def clean_html(text):
    # Удаляем HTML-теги и тому подобное
    if not isinstance(text, str):
        return text
    text = re.sub(r'<[^>]+>', ' ', text)       # теги <...>
    text = re.sub(r'&[a-zA-Z]+?;', ' ', text)  # сущности &nbsp;, &gt;, &amp; и тд
    return text

def clean_markdown(text):
    # Удаляем простую Markdown-разметку:
      # - Ссылки [текст](url) → текст
      # - Заголовки #, ##, ###
      # - Жирный/курсив: **...**, *...*, __...__, _..._
      # - Inline-код `...`
      # - Маркеры списка "- " или "* "
    if not isinstance(text, str):
        return text
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)   # [текст](url) → текст
    text = re.sub(r'(?m)^[#]{1,6}\s*', '', text)          # заголовки #, ## и т.п.
    text = re.sub(r'(\*\*|\*|__|_)(.*?)\1', r'\2', text)  # **жирное**, *курсив*
    text = re.sub(r'`([^`]+)`', r'\1', text)              # `код`
    text = re.sub(r'(?m)^\s*[-*]\s+', '', text)           # строки, начинающиеся с "- " или "* "
    return text

def normalize_ellipses_and_quotes(text):
    # Приводим многоточие и кавычки к единому виду:
      # - Три и более точек → один знак U+2026 «…»
      # - ёлочки (« » и “ ”) → обычные двойные кавычки "
      # - Одинарные ‘ ’ → апостроф '
    if not isinstance(text, str):
        return text
    text = re.sub(r'\.{3,}', '…', text)
    text = text.replace('«', '"').replace('»', '"')
    text = text.replace('“', '"').replace('”', '"')
    text = text.replace('‘', "'").replace('’', "'")
    return text

def normalize_comparisons(text):
    # Преобразуем символы «≤» → '<=' и «≥» → '>='
    if not isinstance(text, str):
        return text
    return text.replace('≤', '<=').replace('≥', '>=')

def normalize_fractions(text):
    # Заменяет одиночные юникод-фракции (½, ⅓, ¾ и т.п.) на эквиваленты "1/2", "1/3", "3/4" и тд
    if not isinstance(text, str):
        return text
    frac_map = {
        '½': '1/2', '⅓': '1/3', '⅔': '2/3', '¼': '1/4', '¾': '3/4',
        '⅕': '1/5', '⅖': '2/5', '⅗': '3/5', '⅘': '4/5',
        '⅙': '1/6', '⅚': '5/6', '⅐': '1/7', '⅛': '1/8', '⅜': '3/8',
        '⅝': '5/8', '⅞': '7/8', '⅑': '1/9', '⅒': '1/10'
    }
    for uni_frac, ascii_frac in frac_map.items():
        text = text.replace(uni_frac, ascii_frac)
    return text

def normalize_special_symbols(text):
    # Убираем диакритические знаки через NFKD-нормализацию
    if not isinstance(text, str):
        return text
    normalized = unicodedata.normalize('NFKD', text)
    return ''.join(ch for ch in normalized if not unicodedata.combining(ch))

def clean_text(text):
    # Применяем всю цепочку очистки к строке:
      # 1. remove_control_symbols
      # 2. split_on_case
      # 3. clean_html
      # 4. clean_markdown
      # 5. normalize_ellipses_and_quotes
      # 6. normalize_comparisons
      # 7. normalize_fractions
      # 8. normalize_special_symbols
      # 9. сводим повторяющиеся пробелы к одному, strip()
    # Возвращаем None, если всё равно пусто или если вход не строка.

    if not isinstance(text, str):
        return None
    text = remove_control_symbols(text)
    text = split_on_case(text)
    text = clean_html(text)
    text = clean_markdown(text)
    text = normalize_ellipses_and_quotes(text)
    text = normalize_comparisons(text)
    text = normalize_fractions(text)
    text = normalize_special_symbols(text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text if text else None

Теперь нам нужна функция для выделения instruction, task и text. Эвристически делим condition.text на три части: instruction, task, text. В SLAVA1.0 нашли в общих чертах определения и сделали вывод что: instruction — строка, содержащая инструкции (что нужно сделать), оканчивается первым из символов '.', '?' или '!'; task — формулировка задачи, идущая сразу после инструкции, до первого ':' или ';'; text — основной контекст или фраза, идущая после ':' или ';'

    Алгоритм:
      1) Удаляем невидимые символы (remove_control_symbols).
      2) Разрываем «склеенные» слова по регистру (split_on_case).
      3) Удаляем HTML и Markdown (clean_html + clean_markdown), сохраняя знаки препинания.
      4) Пытаемся выделить instruction:
         a) Ищем ключевое слово-инструкцию в начале строки:
            ^\s*(Найдите|Определите|Вычислите|Докажите|Укажите|Сколько|Чему|Объясните|Соотнесите|Выберите|Установите|Запишите|Проанализируйте|Опишите|Назовите|Составьте|Дайте определение|Прочитайте|Просмотрите|Answer|Choose|Match|Fill|Complete|Listen| Вы услышите|Прослушайте)\b 
         b) Если найдём, берём от начала до первого из '?', '!', '.' включительно
            как instruction. Остаток (rest) — всё после этого символа.
         c) Если не найдём ключевого слова, instruction = None, rest = весь очищенный текст.
      5) Разбиваем rest на task и text:
         a) Если rest содержит первый из разделителей ':' или ';' или '—' или '–':
            • До и включая этот разделитель = task.
            • Всё после = text.
         b) Иначе task = rest, text = None.
      6) Очищаем instruction, task, text через clean_text и возвращаем кортеж.

    Возвращаем (instruction, task, text), где каждое поле — строка или None.

In [14]:
def derive_task_fields(condition_text):
    if not isinstance(condition_text, str):
        return None, None, None

    raw = condition_text.strip()
    if not raw:
        return None, None, None

    # Удаляем невидимые символы
    raw = remove_control_symbols(raw)
    # Разрываем «склеенные» слова по регистру
    raw = split_on_case(raw)
    # Удаляем HTML и Markdown, сохраняя знаки препинания
    temp = clean_html(raw)
    temp = clean_markdown(temp)

    # Выделяем instruction, если она начинается с ключевого слова
    instr_pattern = re.compile(
        r'^\s*(?P<verb>(?:Найдите|Определите|Вычислите|Докажите|Укажите|Сколько|Чему|'
        r'Объясните|Соотнесите|Выберите|Установите|Запишите|Проанализируйте|Опишите|Назовите|'
        r'Составьте|Дайте определение|Прочитайте|Просмотрите|Answer|Choose|Match|Fill|Complete|'
        r'Listen|Вы услышите|Прослушайте))\b', flags=re.IGNORECASE
    )
    m = instr_pattern.search(temp)
    if m:
        start_idx = m.start()
        end_idx = len(temp)
        for sep in ['?', '!', '.']:
            idx = temp.find(sep, start_idx)
            if idx != -1:
                end_idx = idx + 1
                break
        instruction_raw = temp[:end_idx].strip()
        rest = temp[end_idx:].strip()
    else:
        instruction_raw = None
        rest = temp

    # Разделяем rest на task и text
    if rest:
        sep_index = None
        for sep in [':', ';', '—', '–']:
            idx = rest.find(sep)
            if idx != -1:
                sep_index = idx
                break
        if sep_index is not None:
            task_raw = rest[:sep_index + 1].strip()
            text_raw = rest[sep_index + 1:].strip()
        else:
            task_raw = rest
            text_raw = None
    else:
        task_raw = None
        text_raw = None

    return clean_text(instruction_raw), clean_text(task_raw), clean_text(text_raw)

Функция для извлечения вариантов ответов из текста. 
Ищем в condition_text варианты ответов, если они не заданы в JSON-полях 'options'/'choices'.

    Поддерживаем форматы (после удаления невидимых символов и split_on_case):
      1) Числовые с круглой скобкой: "1) Текст", "2) Текст", ...
      2) Латинские буквы с "." или ")": "A. Текст", "B) Текст", ...
      3) Кириллические буквы с "." или ")": "А. Текст", "Б) Текст", ...

    Алгоритм:
      - Удаляем невидимые символы (remove_control_symbols) и split_on_case.
      - Разбиваем по числовым меткам r'(\d+\))', собираем пары (число, текст_до_следующего_номера).
      - Если найдено ≥2 таких пар, сортируем по числу и возвращаем список текстов.
      - Иначе ищем латинские метки r'(?:^|\n)\s*([A-Z])[\.\)]\s*([^\n]+)' (≥2 → сортируем по букве).
      - Иначе ищем кириллические r'(?:^|\n)\s*([А-ЯЁ])[\.\)]\s*([^\n]+)' (≥2 → сортируем).
      - Иначе возвращаем [].

In [15]:
def extract_options_from_text(condition_text):
    if not isinstance(condition_text, str):
        return []

    raw = remove_control_symbols(condition_text)
    raw = split_on_case(raw)
    text = raw

    # Числовые варианты "1) Текст", "2) Текст"
    tokens = re.split(r'(\d+\))', text)
    numeric_opts = []
    for i in range(len(tokens)):
        if re.fullmatch(r'\d+\)', tokens[i]):
            num = int(tokens[i][:-1])
            txt = tokens[i+1].strip() if i+1 < len(tokens) else ''
            clean_txt = clean_text(txt)
            if clean_txt:
                numeric_opts.append((num, clean_txt))
    if len(numeric_opts) >= 2:
        sorted_opts = sorted(numeric_opts, key=lambda x: x[0])
        return [opt for (_, opt) in sorted_opts]

    # Латинские буквенные "A. Текст" или "A) Текст"
    pattern_latin = re.compile(r'(?:^|\n)\s*([A-Z])[\.\)]\s*([^\n]+)')
    latin_matches = pattern_latin.findall(text)
    if latin_matches and len(latin_matches) >= 2:
        sorted_latin = sorted(latin_matches, key=lambda x: ord(x[0]))
        return [clean_text(m[1]) for m in sorted_latin if clean_text(m[1])]

    # Кириллические буквенные "А. Текст", "Б) Текст"
    pattern_cyril = re.compile(r'(?:^|\n)\s*([А-ЯЁ])[\.\)]\s*([^\n]+)')
    cyril_matches = pattern_cyril.findall(text)
    if cyril_matches and len(cyril_matches) >= 2:
        sorted_cyril = sorted(cyril_matches, key=lambda x: ord(x[0]))
        return [clean_text(m[1]) for m in sorted_cyril if clean_text(m[1])]

    return []

Функция для определения типа задачи. 
Определяет тип задачи по следующим правилам:

      1) Если в тексте (condition.text + solution.text) есть "соответств", "match", "pair", "сопостав" → 'соответствие'.
      2) Если есть "множественн(ый|ая|ые) выбор", "выберите несколько", "choose several",
         "multiple choice.*multiple answers" → 'множественный выбор'.
      3) Если есть фразы об аудировании ("Вы услышите", "Прослушайте", "Listen", "listening") → 'аудирование'.
      4) Если cond.images ≥ 2 и raw_answer:
         a) Если raw_answer содержит ',' или ';' или '|' или ' и ' → 'множественный выбор'.
         b) Если raw_answer — строка из ≥2 цифр без разделителей → 'множественный выбор'.
         c) Если raw_answer — одиночная цифра и 1 ≤ int(raw_answer) ≤ len(cond.images) → 'выбор ответа (один)'.
         d) Иначе → 'соответствие'.
      5) Если найдено ≥2 вариантов в options_list и raw_answer:
         a) Если raw_answer содержит ',' или ';' или '|' или ' и ' → 'множественный выбор'.
         b) Если raw_answer — одиночная цифра или одиночная буква (A–Z или А–Я) → 'выбор ответа (один)'.
         c) Иначе → 'соответствие'.
      6) Если <2 вариантов и raw_answer непустая → 'открытый ответ'.
      7) Иначе → 'неизвестно'.

In [16]:
def detect_task_type(record, instruction, task, text, options_list, raw_answer):
    cond_text = record.get('condition', {}).get('text', '') or ''
    sol_text = record.get('solution', {}).get('text', '') or ''
    combined = (cond_text + ' ' + sol_text).lower()

    # Соответствие
    if re.search(r'соответств|соотнесит|match|pair|сопостав', combined, re.IGNORECASE):
        return 'соответствие'

    # Множественный выбор
    if re.search(
        r'множественн(?:ый|ая|ые) выбор|выберите несколько|choose several|multiple choice.*multiple answers',
        combined, re.IGNORECASE
    ):
        return 'множественный выбор'

    # Аудирование
    if re.search(r'вы услышите|прослушайте|listen|listening|аудио|слушайте', combined, re.IGNORECASE):
        return 'аудирование'

    # Если есть изображения (chem tasks) и raw_answer
    cond_imgs = record.get('condition', {}).get('images') or []
    if len(cond_imgs) >= 2 and raw_answer and raw_answer.strip():
        ans = raw_answer.strip()
        # Несколько через разделители → множественный выбор
        if re.search(r'[,;\|]|(?:\bи\b)', ans):
            return 'множественный выбор'
        # Несколько цифр подряд без разделителей → множественный выбор
        if re.fullmatch(r'\d{2,}', ans):
            return 'множественный выбор'
        # Одиночная цифра → выбор ответа (один)
        if re.fullmatch(r'\d+', ans):
            idx = int(ans)
            if 1 <= idx <= len(cond_imgs):
                return 'выбор ответа (один)'
        # Иначе → соответствие
        return 'соответствие'

    # Если нашли явные варианты (options_list) ≥2
    valid_opts = [o for o in options_list if o]
    if len(valid_opts) >= 2 and raw_answer and raw_answer.strip():
        ans = raw_answer.strip()
        # Несколько через разделители → множественный выбор
        if re.search(r'[,;\|]|(?:\bи\b)', ans):
            return 'множественный выбор'
        # Одиночная цифра или буква → выбор ответа (один)
        if re.fullmatch(r'\d+', ans):
            idx = int(ans)
            if 1 <= idx <= len(valid_opts):
                return 'выбор ответа (один)'
        if re.fullmatch(r'[A-Za-zА-Яа-яЁё]', ans):
            return 'выбор ответа (один)'
        # Иначе → соответствие
        return 'соответствие'

    # Если <2 вариантов и raw_answer непустая → открытый ответ
    if (not valid_opts or len(valid_opts) < 2) and raw_answer and raw_answer.strip():
        return 'открытый ответ'

    # 7) Иначе → неизвестно
    return 'неизвестно'

Функция для формирования comment. 
Формирует поле comment:

      1) Если есть изображения в condition.images или solution.images → 'contains image(s)'.
      2) Если есть фразы об аудировании → 'audio task'.
      3) Иначе → None.

In [17]:
def form_comment(record, existing_comment):
    parts = []
    raw_comment = existing_comment or ''
    if raw_comment.strip():
        parts.append(clean_text(raw_comment))

    cond_imgs = record.get('condition', {}).get('images') or []
    sol_imgs = record.get('solution', {}).get('images') or []
    img_urls = cond_imgs + sol_imgs
    if img_urls:
        parts.append(' '.join(img_urls))

    if parts:
        return ' '.join(parts)

    cond_text = record.get('condition', {}).get('text', '') or ''
    sol_text = record.get('solution', {}).get('text', '') or ''
    combined = (cond_text + ' ' + sol_text).lower()
    if re.search(r'вы услышите|прослушайте|listen|listening|аудио|слушайте', combined, re.IGNORECASE):
        return 'audio task'

    return None

Пробуем обработку одной записи и смотрим, что не так, дорабатываем варианты. Преобразует одну запись JSON в словарь с полями: 'id', 'subject', 'type', 'instruction', 'task', 'text', 'option_1' … 'option_9', 'outputs', 'source', 'comment'.

    Алгоритм:
      1. id и subject
      2. derive_task_fields → instruction, task, text
      3. Формирование вариантов:
         a) raw_opts = record.get('options') или record.get('choices') — берем, если len ≥2.
         b) Иначе, если cond.images ≥2 (chem), используем URL картинок как opts.
         c) Иначе — извлекаем из текста через extract_options_from_text.
         d) В результате берем первые 9 вариантов, дополняем None.
      4. Дополнительная чистка опций:
         - Определяем список глаголов-инструкций.
         - В каждой опции ищем первое вхождение любого такого глагола как отдельного слова.
         - Если найдено, разделяем опцию на base = текст до глагола, extra = от глагола до конца.
           • base остаётся в options.
           • extra добавляется к task (через пробел).
      5. outputs = clean_text(record['answer'])
      6. type = detect_task_type(...)
      7. source = extract_source(record['url'])
      8. comment = form_comment(...)

In [18]:
def process_record(record):
    task_id = record.get('id')
    subj = record.get('subject') or record.get('topic_name') or record.get('category_name')
    subject = clean_text(subj) if subj else None

    cond = record.get('condition', {}) or {}
    cond_text_raw = cond.get('text', '') or ''
    instruction, task, text = derive_task_fields(cond_text_raw)

    # Формирование первоначальных вариантов
    opts = []
    raw_opts = record.get('options') or record.get('choices') or []
    if isinstance(raw_opts, list) and len(raw_opts) >= 2:
        valid_opts = [clean_text(o) for o in raw_opts if isinstance(o, str) and o.strip()]
        if len(valid_opts) >= 2:
            opts = valid_opts[:9] + [None] * max(0, 9 - len(valid_opts))

    # Если raw_opts не дал ≥2, смотрим изображения
    cond_imgs = cond.get('images') or []
    if (not opts or sum(1 for o in opts if o) < 2) and len(cond_imgs) >= 2:
        img_opts = [url for url in cond_imgs[:9]]
        opts = img_opts + [None] * max(0, 9 - len(img_opts))

    # Если всё ещё <2, извлекаем из текста
    if not opts or sum(1 for o in opts if o) < 2:
        extracted = extract_options_from_text(cond_text_raw)
        if extracted and len(extracted) >= 2:
            valid_opts = [o for o in extracted if o]
            opts = valid_opts[:9] + [None] * max(0, 9 - len(valid_opts))

    if not opts or sum(1 for o in opts if o) < 2:
        opts = [None] * 9

    # Дополнительная чистка опций
    extra_task_fragment = ''
    cleaned_opts = []
    # Список глаголов-инструкций, после которых начинаются лишние предложения
    instr_verbs = [
        'Запишите', 'Найдите', 'Определите', 'Вычислите', 'Докажите', 'Укажите', 'Сколько', 'Чему',
        'Объясните', 'Соотнесите', 'Выберите', 'Установите',
        'Answer', 'Choose', 'Match', 'Fill', 'Complete', 'Listen'
    ]
    # Компилируем паттерн для поиска любого из слов-инструкций как отдельного слова
    instr_pattern = re.compile(
        r'\b(' + '|'.join(map(re.escape, instr_verbs)) + r')\b',
        flags=re.IGNORECASE
    )

    for opt in opts:
        if not opt:
            cleaned_opts.append(None)
            continue

        # Находим первое вхождение глагола-инструкции
        m = instr_pattern.search(opt)
        if m:
            start_idx = m.start()
            base = opt[:start_idx].rstrip(' .!?')
            extra = opt[start_idx:].strip()
            # Добавляем extra к task
            extra_task_fragment += ' ' + extra
            cleaned_opts.append(clean_text(base))
        else:
            cleaned_opts.append(opt)

    # Обновляем opts
    opts = [o if o else None for o in cleaned_opts]

    # Обновляем task, добавляя extra_task_fragment (если есть)
    if extra_task_fragment:
        combined_task = (task or '') + ' ' + extra_task_fragment
        task = clean_text(combined_task)

    # Формируем словарь option_1 … option_9
    options_dict = {f'option_{i+1}': opts[i] for i in range(9)}

    # outputs
    raw_answer = record.get('answer', '') or ''
    outputs = clean_text(raw_answer)

    # type
    task_type = detect_task_type(record, instruction, task, text, opts, raw_answer or '')

    # source
    source = record.get('url') or None

    # 8. comment
    existing_comment = record.get('comment') or ''
    comment = form_comment(record, existing_comment)

    return {
        'id': task_id,
        'subject': subject,
        'type': task_type,
        'instruction': instruction,
        'task': task,
        'text': text,
        **options_dict,
        'outputs': outputs,
        'source': source,
        'comment': comment
    }

Пишем функцию для валидации для непустого текста условия (condition.text), так как у нас много открытых заданий в базе, где варианты ответа не предусмотрены, то если мы будем отбрасывать все задание, где не нашлось 2 варинта ответа, это большая часть нашего датафрейма
   

In [19]:
def is_valid_record(record):
    cond_text = record.get('condition', {}).get('text', '') or ''
    return bool(cond_text.strip())


Строим DataFrame из JSON. Загружаем JSON по пути json_path, функция обрабатывает каждую запись через process_record, отбрасывает записи, где 'task' = None, и возвращает pandas.DataFrame с колонками:['id', 'subject', 'type', 'instruction', 'task', 'text', 'option_1', …, 'option_9', 'outputs', 'source', 'comment']

In [20]:
def build_dataframe(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed = []
    for rec in data:
        if not is_valid_record(rec):
            continue
        proc = process_record(rec)
        if proc and proc['task']:
            processed.append(proc)

    columns = [
        'id', 'subject', 'type', 'instruction', 'task', 'text'
    ] + [f'option_{i}' for i in range(1, 10)] + ['outputs', 'source', 'comment']

    df = pd.DataFrame(processed, columns=columns)
    return df

Смотрим, что получилось

In [21]:
if __name__ == '__main__':
    json_file = 'final_ochka_dirty.json'
    df = build_dataframe(json_file)

In [22]:
df

,id,subject,type,instruction,task,text,option_1,option_2,option_3,option_4,option_5,option_6,option_7,option_8,option_9,outputs,source,comment
0,15619,inf,открытый ответ,None,На рисунке справа схема дорог Н-ского раиона и...,None,None,None,None,None,None,None,None,None,None,35,https://inf-ege.sdamgia.ru/problem?id=15619,https://inf-ege.sdamgia.ru/get_file?id=112886
1,15843,inf,открытый ответ,None,На рисунке слева изображена схема дорог Н-ског...,None,None,None,None,None,None,None,None,None,None,67,https://inf-ege.sdamgia.ru/problem?id=15843,https://inf-ege.sdamgia.ru/get_file?id=112884
2,15971,inf,открытый ответ,None,На рисунке схема дорог изображена в виде графа...,None,None,None,None,None,None,None,None,None,None,26,https://inf-ege.sdamgia.ru/problem?id=15971,https://inf-ege.sdamgia.ru/get_file?id=112887
3,16030,inf,открытый ответ,None,На рисунке слева изображена схема дорог Н-ског...,None,None,None,None,None,None,None,None,None,None,26,https://inf-ege.sdamgia.ru/problem?id=16030,https://inf-ege.sdamgia.ru/get_file?id=112894
4,23901,inf,открытый ответ,None,На рисунке слева изображена схема дорог N-ског...,None,None,None,None,None,None,None,None,None,None,45|П4П5,https://inf-ege.sdamgia.ru/problem?id=23901,https://inf-ege.sdamgia.ru/get_file?id=112898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12501,2510,fr,неизвестно,None,Vous travaillez avec votre ami(e) sur le proje...,• ; expliquez le choix des photos pour le proj...,None,None,None,None,None,None,None,None,None,None,https://fr-ege.sdamgia.ru/problem?id=2510,https://fr-ege.sdamgia.ru/get_file?id=17636
12502,2513,fr,неизвестно,None,Vous travaillez avec votre ami(e) sur le proje...,• ; expliquez le choix des photos pour le proj...,None,None,None,None,None,None,None,None,None,None,https://fr-ege.sdamgia.ru/problem?id=2513,https://fr-ege.sdamgia.ru/get_file?id=84783
12503,2515,fr,неизвестно,None,Vous travaillez avec votre ami(e) sur le proje...,• expliquez le choix des photos pour le projet...,None,None,None,None,None,None,None,None,None,None,https://fr-ege.sdamgia.ru/problem?id=2515,https://fr-ege.sdamgia.ru/get_file?id=17640
12504,2516,fr,неизвестно,None,Vous travaillez avec votre ami(e) sur le proje...,• expliquez le choix des photos pour le projet...,None,None,None,None,None,None,None,None,None,None,https://fr-ege.sdamgia.ru/problem?id=2516,https://fr-ege.sdamgia.ru/get_file?id=17632


Выгружаем наш датасет 

In [23]:
df.to_csv('slava2.0_full.csv', index=False, encoding='utf-8')

Видим, что возникает проблема с тем, что у нас в базе много заданий с открытым ответом, поэтому столбцы option чаще всего пустые. Для удобства дальнейшей работы решено преобразовать код, чтобы датафрейм выглядел более аккуратно

Для этого переписала несколько функций, но логика их работы такая же 

In [24]:
def detect_tasktype(record, task, options_list, raw_answer):
    cond_text = record.get('condition', {}).get('text', '') or ''
    sol_text = record.get('solution', {}).get('text', '') or ''
    combined = (cond_text + ' ' + sol_text).lower()

    if re.search(r'соответств|соотнесит|match|pair|сопостав', combined, re.IGNORECASE):
        return 'соответствие'
    if re.search(r'множественн(?:ый|ая|ые) выбор|выберите несколько|choose several|multiple choice.*multiple answers',
                 combined, re.IGNORECASE):
        return 'множественный выбор'
    if re.search(r'вы услышите|прослушайте|listen|listening|аудио|слушайте', combined, re.IGNORECASE):
        return 'аудирование'

    cond_imgs = record.get('condition', {}).get('images') or []
    if len(cond_imgs) >= 2 and raw_answer and raw_answer.strip():
        ans = raw_answer.strip()
        if re.search(r'[,;\|]|(?:\bи\b)', ans):
            return 'множественный выбор'
        if re.fullmatch(r'\d{2,}', ans):
            return 'множественный выбор'
        if re.fullmatch(r'\d+', ans):
            idx = int(ans)
            if 1 <= idx <= len(cond_imgs):
                return 'выбор ответа (один)'
        return 'соответствие'

    valid_opts = [o for o in options_list if o]
    if len(valid_opts) >= 2 and raw_answer and raw_answer.strip():
        ans = raw_answer.strip()
        if re.search(r'[,;\|]|(?:\bи\b)', ans):
            return 'множественный выбор'
        if re.fullmatch(r'\d+', ans):
            idx = int(ans)
            if 1 <= idx <= len(valid_opts):
                return 'выбор ответа (один)'
        if re.fullmatch(r'[A-Za-zА-Яа-яЁё]', ans):
            return 'выбор ответа (один)'
        return 'соответствие'

    if (not valid_opts or len(valid_opts) < 2) and raw_answer and raw_answer.strip():
        return 'открытый ответ'

    return 'неизвестно'

In [25]:
def processrecord(record):
    # id и subject
    task_id = record.get('id')
    subj = record.get('subject') or record.get('topic_name') or record.get('category_name')
    subject = clean_text(subj) if subj else None

    # task = весь текст condition.text (без разбиения)
    cond = record.get('condition', {}) or {}
    cond_text_raw = cond.get('text', '') or ''
    task = clean_text(cond_text_raw)

    # правильный ответ
    raw_answer = record.get('answer', '') or ''
    outputs = clean_text(raw_answer)

    # source = прямая ссылка на url
    source = record.get('url') or None

    # comment с ссылками на изображения
    existing_comment = record.get('comment') or ''
    comment = form_comment(record, existing_comment)

    # попытка собрать явные варианты для type
    raw_opts = record.get('options') or record.get('choices') or []
    valid_opts = []
    if isinstance(raw_opts, list):
        valid_opts = [clean_text(o) for o in raw_opts if isinstance(o, str) and o.strip()]

    # type
    task_type = detect_tasktype(record, task, valid_opts, raw_answer or '')

    return {
        'id': task_id,
        'subject': subject,
        'type': task_type,
        'task': task,
        'outputs': outputs,
        'source': source,
        'comment': comment
    }

In [26]:
def builddataframe(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed = []
    for rec in data:
        if not is_valid_record(rec):
            continue
        proc = processrecord(rec)
        if proc and proc['task']:
            processed.append(proc)

    columns = ['id', 'subject', 'type', 'task', 'outputs', 'source', 'comment']
    df = pd.DataFrame(processed, columns=columns)
    return df

if __name__ == '__main__':
    df = builddataframe(json_file)

In [27]:
df

,id,subject,type,task,outputs,source,comment
0,15619,inf,открытый ответ,На рисунке справа схема дорог Н-ского раиона и...,35,https://inf-ege.sdamgia.ru/problem?id=15619,https://inf-ege.sdamgia.ru/get_file?id=112886
1,15843,inf,открытый ответ,На рисунке слева изображена схема дорог Н-ског...,67,https://inf-ege.sdamgia.ru/problem?id=15843,https://inf-ege.sdamgia.ru/get_file?id=112884
2,15971,inf,открытый ответ,На рисунке схема дорог изображена в виде графа...,26,https://inf-ege.sdamgia.ru/problem?id=15971,https://inf-ege.sdamgia.ru/get_file?id=112887
3,16030,inf,открытый ответ,На рисунке слева изображена схема дорог Н-ског...,26,https://inf-ege.sdamgia.ru/problem?id=16030,https://inf-ege.sdamgia.ru/get_file?id=112894
4,23901,inf,открытый ответ,На рисунке слева изображена схема дорог N-ског...,45|П4П5,https://inf-ege.sdamgia.ru/problem?id=23901,https://inf-ege.sdamgia.ru/get_file?id=112898
...,...,...,...,...,...,...,...
13041,2510,fr,неизвестно,Vous travaillez avec votre ami(e) sur le proje...,None,https://fr-ege.sdamgia.ru/problem?id=2510,https://fr-ege.sdamgia.ru/get_file?id=17636
13042,2513,fr,неизвестно,Vous travaillez avec votre ami(e) sur le proje...,None,https://fr-ege.sdamgia.ru/problem?id=2513,https://fr-ege.sdamgia.ru/get_file?id=84783
13043,2515,fr,неизвестно,Vous travaillez avec votre ami(e) sur le proje...,None,https://fr-ege.sdamgia.ru/problem?id=2515,https://fr-ege.sdamgia.ru/get_file?id=17640
13044,2516,fr,неизвестно,Vous travaillez avec votre ami(e) sur le proje...,None,https://fr-ege.sdamgia.ru/problem?id=2516,https://fr-ege.sdamgia.ru/get_file?id=17632


In [28]:
df.to_csv('slava2.0_pt.csv', index=False, encoding='utf-8')